In [1]:
import sys
sys.path.append('..')
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sfp_nsdsyn import *
import warnings
import itertools
import matplotlib as mpl

warnings.filterwarnings("ignore", category=UserWarning)
pd.options.mode.chained_assignment = None
%load_ext autoreload
%autoreload 2


In [2]:
output_dir='/Volumes/server/Projects/sfp_nsd/derivatives'
results_dir = os.path.join(output_dir, 'sfp_model', 'results_2D')
fig_dir = os.path.join(output_dir, 'figures')
lr = 0.0005
max_epoch = 30000
roi_list = ['V1', 'V2', 'V3']
vsc='pRFsize'
nsd_subj_list = [utils.sub_number_to_string(sn, 'nsdsyn') for sn in np.arange(1,9)]
broderick_sn_list = [1, 6, 7, 45, 46, 62, 64, 81, 95, 114, 115, 121]
broderick_subj_list = [utils.sub_number_to_string(i, 'broderick') for i in broderick_sn_list]
ARGS_2D = ['sub','lr','eph','roi']
roi_pal = [sns.color_palette('dark', 10)[:][k] for k in  [3,2,0]]
roi_pal.insert(0, (0.3,0.3,0.3))
stim_classes=['annulus', 'pinwheel', 'forward spiral','reverse spiral']

# Predict synthetic data based on the model parameters

```python
stim_info='/Volumes/server/Projects/sfp_nsd/natural-scenes-dataset/nsdsyn_stim_description.csv'
stim_info = vis2D.get_w_a_and_w_r_for_each_stim_class(stim_info)

ecc_range = (0, 12)
n_ecc = 121
angle_range = (np.deg2rad(0), np.deg2rad(360))
n_angle = 360
ecc_col = 'eccentricity'
angle_col = 'angle'
synthetic_df = vis2D.make_synthetic_dataframe_for_2D(stim_info,
                                                ecc_range, n_ecc,
                                                angle_range, n_angle,
                                                ecc_col, angle_col)

synthetic_df['local_ori'] = prep.calculate_local_orientation(w_a = synthetic_df['w_a'],
                                                     w_r = synthetic_df['w_r'],
                                                     retinotopic_angle=synthetic_df[angle_col],
                                                     angle_in_radians=True,
                                                     reference_frame='relative')

synthetic_df['local_ori'] = prep.calculate_local_orientation(w_a = synthetic_df['w_a'],
                                                     w_r = synthetic_df['w_r'],
                                                     retinotopic_angle=synthetic_df[angle_col],
                                                     angle_in_radians=True,
                                                     reference_frame='relative')

synthetic_df['Pv'] = synthetic_df.apply(model.get_Pv_row, 
                                        params=final_params.query('sub == "subj01" & vroinames == "V1"'), 
                                        axis=1)



# Load models

In [3]:
pt_file_list = []
for subj, roi in itertools.product(nsd_subj_list, roi_list):
    file_name = os.path.join(output_dir, 'sfp_model', 'results_2D', 'nsdsyn', 'corrected',
                             f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
nsd_params = model.load_all_models(pt_file_list, *ARGS_2D)

nsd_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'nsdsyn', 'precision', f'precision-s_dset-nsdsyn_vs-pRFsize.csv'))
nsd_df = pd.merge(nsd_params, nsd_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])


In [4]:
pt_file_list = []
for subj, roi in itertools.product(broderick_subj_list, ['V1']):
    file_name = os.path.join(output_dir,'sfp_model', 'results_2D', 'broderick','corrected', f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
broderick_params = model.load_all_models(pt_file_list, *ARGS_2D)

broderick_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'broderick', 'precision','corrected', f'precision-s_dset-broderick_vs-pRFsize.csv'))
broderick_df = pd.merge(broderick_params, broderick_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [5]:
broderick_df['dset_type'] = 'Broderick et al. V1'
nsd_df['dset_type'] = nsd_df['vroinames'].apply(lambda x: f'NSD {x}')
final_params = pd.concat((broderick_df, nsd_df), axis=0)

# Load synthetic data

In [8]:
dset_type_list = ['NSD V1', 'NSD V2', 'NSD V3', 'Broderick et al. V1']
tmp = final_params.query('dset_type in @dset_type_list')
PARAMS_2D = ['sigma', 'slope', 'intercept', 'p_1', 'p_2', 'p_3', 'p_4', 'A_1', 'A_2']
weighted_means = utils.calculate_weighted_mean(final_params, PARAMS_2D, 'precision', groupby=['dset_type'])
# Reorder the rows of weighted_means to match dset_type_list
weighted_means.set_index('dset_type', inplace=True)
weighted_means = weighted_means.loc[dset_type_list]
weighted_means = weighted_means.transpose()
weighted_means['average'] = 'weighted'

In [9]:
weighted_means

dset_type,NSD V1,NSD V2,NSD V3,Broderick et al. V1,average
sigma,2.199299,3.797996,4.461167,2.276244,weighted
slope,0.150471,0.171325,0.167584,0.115166,weighted
intercept,0.171367,0.253642,0.202358,0.371720,weighted
p_1,0.080991,0.161812,0.051839,0.070505,weighted
p_2,-0.021897,-0.102574,-0.046833,-0.028079,weighted
p_3,-0.036557,-0.342089,-0.463685,0.057750,weighted
p_4,-0.065930,0.039329,0.042397,-0.004898,weighted
A_1,0.057589,0.046426,0.043957,0.044961,weighted
A_2,-0.019375,-0.024967,-0.012346,-0.004640,weighted


In [ ]:
unweighted_means = utils.calculate_weighted_mean(final_params, PARAMS_2D, None, groupby=['dset_type'])

unweighted_means = unweighted_means.transpose().rename(columns={0: 'NSD V1', 1: 'NSD V2', 2: 'NSD V3'})
unweighted_means = unweighted_means.iloc[1:]
unweighted_means['average'] = 'unweighted'

In [ ]:
final_number_df = pd.concat([weighted_means, unweighted_means]).loc[PARAMS_2D][['average', 'NSD V1', 'NSD V2', 'NSD V3', ]]
final_number_df

,average,NSD V1,NSD V2,NSD V3
sigma,weighted,2.276244,2.199299,3.797996
sigma,unweighted,2.185474,2.213854,3.710788
slope,weighted,0.115166,0.150471,0.171325
slope,unweighted,0.116133,0.143618,0.177348
intercept,weighted,0.37172,0.171367,0.253642
intercept,unweighted,0.361082,0.148713,0.189056
p_1,weighted,0.070505,0.080991,0.161812
p_1,unweighted,0.074682,0.094223,0.159324
p_2,weighted,-0.028079,-0.021897,-0.102574
p_2,unweighted,-0.028865,-0.030134,-0.10177


In [ ]:
pt_file_list = []
for subj, roi in itertools.product(broderick_subj_list, ['V1']):
    file_name = os.path.join(output_dir,'sfp_model', 'results_2D', 'broderick','corrected', f'model-params_lr-{lr}_eph-{max_epoch}_sub-{subj}_roi-{roi}_vs-{vsc}.pt')
    pt_file_list.append(file_name)
broderick_params = model.load_all_models(pt_file_list, *ARGS_2D)

broderick_precision_s =  pd.read_csv(os.path.join(output_dir, 'dataframes', 'broderick', 'precision','corrected', f'precision-s_dset-broderick_vs-pRFsize.csv'))
broderick_df = pd.merge(broderick_params, broderick_precision_s[['sub','vroinames','precision']], on=['sub','vroinames'])

In [ ]:
broderick_df['dset_type'] = 'Broderick et al. V1'
nsd_df['dset_type'] = nsd_df['vroinames'].apply(lambda x: f'NSD {x}')
final_params = pd.concat((broderick_df, nsd_df), axis=0)